<a href="https://colab.research.google.com/github/dmi3eva/araneae/blob/main/p1_spider_analysis/_UTILS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content
%cd drive
%cd My\ Drive
%cd PhD
%cd Paper_01

/content
/content/drive
/content/drive/My Drive
/content/drive/My Drive/PhD
/content/drive/My Drive/PhD/Paper_01


In [3]:
import json
import collections
import matplotlib.pyplot as plt
import sqlite3
import os

### Обращение к БД

Вспомогательное

In [4]:
DB_PATH = "datasets/spider/database"
SCHEMES_PATH = "datasets/spider/tables.json"

In [5]:
def execute_request(db_id, sql):
    db = os.path.join(DB_PATH, db_id, db_id + ".sqlite")
    conn = sqlite3.connect(db)
    conn.text_factory = lambda b: b.decode(errors = 'ignore')
    cursor = conn.cursor()   
    try:
        cursor.execute(sql)
        res = cursor.fetchall()
        return res
    except:
        raise ValueError

In [64]:
REQUEST_MASK = """SELECT \"{column}\" FROM \"{table}\"""" 
def get_values(db, table, column):
  # Возвращает значения из столбца данной таблицы
  aim_request = REQUEST_MASK.format(table=table, column=column)
  print(aim_request)
  try:
    response = execute_request(db, aim_request)
    values = [str(_v[0]) for _v in list(response)]
    return values
  except ValueError:
    print()
    print(f"Problem with {column} in {table} (db = {db}). Request: {aim_request}")
    return []

In [7]:
db_ids = os.listdir(DB_PATH)

In [38]:
with open(SCHEMES_PATH) as table_file:
  schemes = json.load(table_file)

tables = {_s['db_id']: _s['table_names_original'] for _s in schemes}
sorted(list(tables.keys()))

['academic',
 'activity_1',
 'aircraft',
 'allergy_1',
 'apartment_rentals',
 'architecture',
 'assets_maintenance',
 'baseball_1',
 'battle_death',
 'behavior_monitoring',
 'bike_1',
 'body_builder',
 'book_2',
 'browser_web',
 'candidate_poll',
 'car_1',
 'chinook_1',
 'cinema',
 'city_record',
 'climbing',
 'club_1',
 'coffee_shop',
 'college_1',
 'college_2',
 'college_3',
 'company_1',
 'company_employee',
 'company_office',
 'concert_singer',
 'county_public_safety',
 'course_teach',
 'cre_Doc_Control_Systems',
 'cre_Doc_Template_Mgt',
 'cre_Doc_Tracking_DB',
 'cre_Docs_and_Epenses',
 'cre_Drama_Workshop_Groups',
 'cre_Theme_park',
 'csu_1',
 'culture_company',
 'customer_complaints',
 'customer_deliveries',
 'customers_and_addresses',
 'customers_and_invoices',
 'customers_and_products_contacts',
 'customers_campaigns_ecommerce',
 'customers_card_transactions',
 'debate',
 'decoration_competition',
 'department_management',
 'department_store',
 'device',
 'document_management',

In [37]:
columns = {
    _s['db_id']: {
        _table: [] for _table in _s['table_names_original']
    } for _s in schemes
}
for _scheme in schemes:
  db_id = _scheme['db_id']
  table_names = _scheme['table_names_original']
  for _column in _scheme["column_names_original"]:
    column_name = _column[1]
    table_name = table_names[_column[0]]
    if column_name == '*':
      continue
    columns[db_id][table_name].append(column_name)

In [11]:
def get_db_tables(db):
  return tables[db]

In [12]:
def get_db_columns(db, table):
  return columns[db][table]

Сами обращения

In [181]:
db = 'employee_hire_evaluation'
table = 'hiring'
column = 'CountryCode'

In [182]:
result_tables = get_db_tables(db)
result_tables

['employee', 'shop', 'hiring', 'evaluation']

In [183]:
result_columns = get_db_columns(db, table)
result_columns

['Shop_ID', 'Employee_ID', 'Start_from', 'Is_full_time']

In [ ]:
result_values = get_values(db, table, column)
result_values

## Выполнение запроса

In [186]:
sql_request = "SELECT DISTINCT(Start_from) FROM hiring WHERE Is_full_time == 'F'"
result = execute_request(db, sql_request)
result

[('2011',), ('2010',)]